In [11]:
from envs.minimal_jsp_env.util.jsp_generation.random_generator import RandomJSPGenerator, RandomJSPGeneratorPool, RandomJSPGeneratorOperationDistirbution, RandomJSPGeneratorWithJobPool
from envs.minimal_jsp_env.entities import Operation, JSPInstance
import json

### Reader / Writer

In [12]:
def get_job_matrices(jsp_instance: JSPInstance, display: bool = True):
    machine_types = []
    durations = []
    for job in jsp_instance.jobs:
        machine_types.append([operation.machine_type for operation in job])
        durations.append([operation.duration for operation in job])
        if display:
            print([f"({operation.machine_type}, {operation.duration})" for operation in job])
    
    return machine_types, durations

In [13]:
test_instance = RandomJSPGenerator(num_jobs=3, num_operations=3, max_op_duration=9).generate()
test_instance.jobs

[[Operation(job_id=0, op_id=0, machine_type=0, duration=5),
  Operation(job_id=0, op_id=1, machine_type=2, duration=7),
  Operation(job_id=0, op_id=2, machine_type=2, duration=5)],
 [Operation(job_id=1, op_id=0, machine_type=1, duration=4),
  Operation(job_id=1, op_id=1, machine_type=2, duration=3),
  Operation(job_id=1, op_id=2, machine_type=1, duration=6)],
 [Operation(job_id=2, op_id=0, machine_type=1, duration=5),
  Operation(job_id=2, op_id=1, machine_type=1, duration=6),
  Operation(job_id=2, op_id=2, machine_type=1, duration=3)]]

In [34]:
class JSPWriterJSON:
    def write_instance(self, instance: JSPInstance, path: str, id: str):
        machine_types, durations = get_job_matrices(instance, display=False)
        json_file = {
            'machine_types': machine_types, 
            'durations': durations, 
            'num_ops_per_job' : instance.num_ops_per_job,
            'max_op_time': instance.max_op_time,
            'id': id,
            'opt_time': instance.opt_time
            }
        with open(path, 'w') as file:
            json.dump(json_file, file)

In [35]:
class JSPReaderJSON:
    def read_instance(self, path: str) -> JSPInstance:       

        with open(path, 'r') as file:
            input_file = json.load(file)

        jobs = []

        for job_id, (job_machine_types, job_durations) in enumerate(zip(input_file['machine_types'], input_file['durations'])):
            
            operations = []
            for operation_id, (operation_machine_type, operation_duration) in enumerate(zip(job_machine_types, job_durations)):
                operations.append(Operation(job_id, operation_id, operation_machine_type, operation_duration))
            
            jobs.append(operations)


        return JSPInstance(jobs, input_file['num_ops_per_job'], input_file['max_op_time'], input_file['id'], input_file['opt_time'])

In [36]:
JSPWriterJSON().write_instance(test_instance, "test.json", "test")

In [37]:
JSPReaderJSON().read_instance('test.json').jobs

[[Operation(job_id=0, op_id=0, machine_type=0, duration=5),
  Operation(job_id=0, op_id=1, machine_type=2, duration=7),
  Operation(job_id=0, op_id=2, machine_type=2, duration=5)],
 [Operation(job_id=1, op_id=0, machine_type=1, duration=4),
  Operation(job_id=1, op_id=1, machine_type=2, duration=3),
  Operation(job_id=1, op_id=2, machine_type=1, duration=6)],
 [Operation(job_id=2, op_id=0, machine_type=1, duration=5),
  Operation(job_id=2, op_id=1, machine_type=1, duration=6),
  Operation(job_id=2, op_id=2, machine_type=1, duration=3)]]

### Dataset generation

In [45]:
import os

In [38]:
# entropies for datasets with pool_size = 36

entropy0_2 = [0.5, 0.5]
entropy0_3 = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
entropy0_4 = [0.08333333333333333, 0.16666666666666666, 0.25, 0.08333333333333333, 0.4166666666666667]
entropy0_5 = [0.3333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666]
entropy0_6 = [0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.16666666666666666, 0.08333333333333333]
entropy0_7 = [0.05555555555555555, 0.1111111111111111, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.16666666666666666, 0.16666666666666666, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.08333333333333333, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.027777777777777776]
entropy0_8 = [0.16666666666666666, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.08333333333333333, 0.05555555555555555, 0.08333333333333333, 0.027777777777777776]

In [46]:
number_of_instances = 100

In [47]:
for entropy_val in range(2, 9, 1):

    dir_path = f"data/entropy_datasets/entropy0_{entropy_val}/"
    os.makedirs(dir_path, exist_ok=True)

    for i in range(number_of_instances):
        instance = RandomJSPGeneratorOperationDistirbution(num_jobs=6, num_operations=6, max_op_duration=9).generate(eval(f"entropy0_{entropy_val}"))
        file_name = f"entropy0_{entropy_val}_{i}"
        JSPWriterJSON().write_instance(instance, f"{dir_path}/{file_name}", file_name)
        

[0.5, 0.5]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[0.08333333333333333, 0.16666666666666666, 0.25, 0.08333333333333333, 0.4166666666666667]
[0.3333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666]
[0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.16666666666666666, 0.08333333333333333]
[0.05555555555555555, 0.1111111111111111, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.16666666666666666, 0.16666666666666666, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.08333333333333333, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.027777777777777776]
[0.16666666666666666, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.0555